## K - FOLD structure folder

In [ ]:
# Create the working enviornment

#Run this cells 

!python -m venv .venv 

# this to activate the enviroment
!Start .venv\Scripts\activate 

In [ ]:
# Install the ultralytics package from PyPI
!pip install ultralytics

!pip install huggingface_hub
!pip install numpy

!pip install optuna
!pip install hyperopt

!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
 


In [ ]:
#TEST FOR GPU
import torch
print(torch.cuda.is_available())  # Should return true if you have cuda
print(torch.cuda.get_device_name(0))


# K FOLD

This is the code used for implementing k-fold cross-validation. It assumes that the data has not been changed and follow the same structure of the original repository (data/dataset/imgs and data/dataset/labels). However, the code may need some adjustments.

In [ ]:
from pathlib import Path

dataset_path = Path("./data/MangaFD")  # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*labels/*.txt"))  # all data in 'labels'


In [ ]:
import yaml

yaml_file = Path().resolve()/"k_fold.yaml"   # your data YAML with data directories and names dictionary
with open(yaml_file, "r", encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]
cls_idx = sorted(classes.keys())


In [ ]:
import pandas as pd

index = [label.stem for label in labels]  # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=index)

In [ ]:
#remember to delete the past split of the dataset before running this code

from collections import Counter

for label in labels:
    lbl_counter = Counter()

    with open(label, "r") as lf:
        lines = lf.readlines()

    for line in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(line.split(" ")[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`

In [ ]:
from sklearn.model_selection import KFold

ksplit = 4  # number of splits for KFold
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [ ]:
folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=index, columns=folds)

for i, (train, val) in enumerate(kfolds, start=1):
    folds_df.loc[labels_df.iloc[train].index, f"split_{i}"] = "train"
    folds_df.loc[labels_df.iloc[val].index, f"split_{i}"] = "val"

In [ ]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1e-7)
    fold_lbl_distrb.loc[f"split_{n}"] = ratio

In [ ]:
import datetime

supported_extensions = [".jpg", ".jpeg", ".png"]

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / "images").rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / "fold_cross")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump(
            {
                "path": split_dir.as_posix(),
                "train": "train",
                "val": "val",
                "names": classes,
            },
            ds_y,
        )

In [ ]:
import shutil

for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / "images"
        lbl_to_path = save_path / split / k_split / "labels"

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

In [ ]:
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

In [ ]:
from ultralytics import YOLO

weights_path = Path().resolve()/"yolo11s.pt"
model = YOLO(weights_path, task="detect")

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os
import torch

results = {}

# Define your additional arguments here
batch = 8
project = "kfold_demo"
epochs = 80  # Imposta il numero di epoche desiderato
weights_path = Path().resolve()/"yolo11s.pt"  # Sostituisci con il percorso corretto del file di pesi

# Parametri aggiuntivi da passare al metodo train
optimizer = 'auto'  # Puoi scegliere l'ottimizzatore desiderato
augment = True  # Abilitazione dell'augmentation
amp = True  # Precisione mista
freeze = [0]  # Congelamento del primo livello (se necessario)
patience = 20  # Pazienza per la ricerca del miglior modello
device = "gpu"  # Specifica se usare la GPU

for k in range(ksplit):
    dataset_yaml = ds_yamls[k]
    model = YOLO(weights_path, task="detect")
    
    # Passa tutti i parametri al metodo `train`
    model.train(
        data=dataset_yaml, 
        epochs=epochs, 
        batch=batch, 
        project=project, 
        optimizer=optimizer, 
        augment=augment, 
        amp=amp, 
        plots=True,  # Se vuoi che venga generato il grafico
        freeze=freeze, 
        patience=patience,
        device=device
    )  
    
    # Salva i risultati per ogni fold
    results[k] = model.metrics  # salva le metriche del modello per l'analisi successiva


In [ ]:
!pip install retina-face

# EVALUATION 

In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os

data_path = Path("./models/best_manga109.pt")

# Load the model
model = YOLO(data_path)  

# Dataset as dictionary
data = dataset_path / "validationFD.yaml"  # change if needed

# Evaluate the model's performance on the validation set
val_results = model.val(data=data)

# References



https://docs.ultralytics.com/modes/train/#train-settings
https://docs.ultralytics.com/it
https://docs.ultralytics.com/modes/predict/#inference-arguments
https://docs.ultralytics.com/guides/hyperparameter-tuning/#custom-search-space-example
https://github.com/optuna/optuna
https://docs.ultralytics.com/tasks/detect/#models
https://github.com/sunsmarterjie/yolov12
https://github.com/serengil/retinaface?tab=readme-ov-file
https://github.com/biubug6/Pytorch_Retinaface?tab=readme-ov-file#training   implementazione pytorch
